In [1]:
import os
import mujoco
import math
import numpy
import mediapy as media
import matplotlib.pyplot as plt

In [2]:
width = 640
height = 480

In [3]:
# animal = 'beetle'
animal = 'turtle'

In [4]:
beetle_template = """
<mujoco>
  <option>
     <flag gravity="enable" contact="enable" />
  </option>
  <compiler angle="degree" />
    <visual><global offwidth="{width}" offheight="{height}" /></visual>    

  <worldbody>
    <light name="top" pos="0 0 1"/>
    <camera name="world" mode="fixed" pos="0 -.5 .05" axisangle="1 0 0 90"/>
    <camera name="target" mode="targetbody" target="trunk" pos="0 -.5 .15" axisangle="1 0 0 90" />

    
    <body name="floor" pos="0 0 0">
      <geom name="floor" pos="0 0 0" size="1 1 .1" type="plane" rgba="1 0.83 0.61 0.5"/>
    </body>

    <body name="trunk" pos="0 0 .1">
        <joint type="free"/>
        <geom type="box" size=".05 .05 .003" pos="0 0 0" rgba="1 0 0 1"/>
        
      <body name="leg1_l1" pos=".03 -.05 0">
          <joint name="leg1_j1" type="hinge" axis="1 0 0" pos="0 0 0" stiffness="{k:e}" damping="{b:e}" limited="true" range="-179 179"/>
          <geom type="box" size=".02 .01 .001" pos="0 -.01 0" rgba="1 0 0 1"/>
          <body name="leg1_l2" pos=".02 0 0">
            <joint name="leg1_j2" type="hinge" axis="0 1 0"  stiffness="{k:e}" damping="{b:e}"  limited="true" range="-179 179"/>
            <geom type="box" size=".02 .01 .001" pos=".02 -.01 0" rgba="1 0 0 1"/>
            <body name="leg1_l4" pos=".04 0 0">
              <joint name="leg1_j3" type="hinge" axis="0 1 0"  stiffness="{k:e}" damping="{b:e}"  limited="true" range="-179 179"/>
              <geom type="box" size=".02 .01 .001" pos=".02 -.01 0" rgba="1 0 0 1"/>
              <geom type="box" size=".01 .01 .001" pos=".01 .01 0" rgba="1 0 0 1"/>

            </body>
          </body>
          <body name="leg1_l3" pos="-.02 0 0">
            <joint name="leg1_j4" type="hinge" axis="0 1 0"  stiffness="{k:e}" damping="{b:e}"  limited="true" range="-179 179"/>
            <geom type="box" size=".02 .01 .001" pos="-.02 -.01 0" rgba="1 0 0 1"/>
            <body name="leg1_l4_2" pos="-.04 0 0">
              <joint name="leg1_j5" type="hinge" axis="0 1 0"  stiffness="{k:e}" damping="{b:e}"  limited="true" range="-179 179"/>
              <geom type="box" size=".02 .01 .001" pos="-.02 -.01 0" rgba="1 0 0 1"/>
            </body>
          </body>
      </body>

      <body name="leg2_l1" pos=".03 .05 0">
          <joint name="leg2_j1" type="hinge" axis="-1 0 0" pos="0 0 0" stiffness="{k:e}" damping="{b:e}" limited="true" range="-179 179"/>
          <geom type="box" size=".02 .01 .001" pos="0 .01 0" rgba="1 0 0 1"/>
          <body name="leg2_l2" pos=".02 0 0">
            <joint name="leg2_j2" type="hinge" axis="0 1 0"  stiffness="{k:e}" damping="{b:e}"  limited="true" range="-179 179"/>
            <geom type="box" size=".02 .01 .001" pos=".02 .01 0" rgba="1 0 0 1"/>
            <body name="leg2_l4" pos=".04 0 0">
              <joint name="leg2_j3" type="hinge" axis="0 1 0"  stiffness="{k:e}" damping="{b:e}"  limited="true" range="-179 179"/>
              <geom type="box" size=".02 .01 .001" pos=".02 .01 0" rgba="1 0 0 1"/>
              <geom type="box" size=".01 .01 .001" pos=".01 -.01 0" rgba="1 0 0 1"/>

            </body>
          </body>
          <body name="leg2_l3" pos="-.02 0 0">
            <joint name="leg2_j4" type="hinge" axis="0 1 0"  stiffness="{k:e}" damping="{b:e}"  limited="true" range="-179 179"/>
            <geom type="box" size=".02 .01 .001" pos="-.02 .01 0" rgba="1 0 0 1"/>
            <body name="leg2_l4_2" pos="-.04 0 0">
              <joint name="leg2_j5" type="hinge" axis="0 1 0"  stiffness="{k:e}" damping="{b:e}"  limited="true" range="-179 179"/>
              <geom type="box" size=".02 .01 .001" pos="-.02 .01 0" rgba="1 0 0 1"/>
            </body>
          </body>
      </body>
        <geom type="sphere" size=".01" pos="-.05 .05 -.01" rgba="1 0 0 1"/>
        <geom type="sphere" size=".01" pos="-.05 -.05 -.01" rgba="1 0 0 1"/>
        <geom type="sphere" size=".01" pos=".05 .05 -.01" rgba="1 0 0 1"/>
        <geom type="sphere" size=".01" pos=".05 -.05 -.01" rgba="1 0 0 1"/>

    </body>
  </worldbody>
  <actuator>
    <position name="m1" joint="leg1_j1"/>
    <position name="m2" joint="leg1_j2"/>
    <position name="m3" joint="leg2_j1"/>
    <position name="m4" joint="leg2_j2"/>
  </actuator>
  
  <equality>
    <weld name="weld1" active="true" body1="leg1_l4" body2="leg1_l4_2" anchor="-.02 0 0" relpose=".02 0 0 1 0 0 0" solimp=".90 .99 .01" solref=".001 1"/>
    <weld name="weld2" active="true" body1="leg2_l4" body2="leg2_l4_2" anchor="-.02 0 0" relpose=".02 0 0 1 0 0 0" solimp=".90 .99 .01" solref=".001 1"/>
  </equality>  
</mujoco>
"""

In [5]:
turtle_template = """
<mujoco>
  <option>
     <flag gravity="enable" contact="enable" />
  </option>
  <compiler angle="degree" />
    <visual><global offwidth="{width}" offheight="{height}" /></visual>    

  <worldbody>
    <light name="top" pos="0 0 1"/>
    <camera name="world" mode="fixed" pos="0 -.5 .05" axisangle="1 0 0 90"/>
    <camera name="target" mode="targetbody" target="trunk" pos="0 -.5 .15" axisangle="1 0 0 90" />

    
    <body name="floor" pos="0 0 0">
      <geom name="floor" pos="0 0 0" size="1 1 .1" type="plane" rgba="1 0.83 0.61 0.5"/>
    </body>

    <body name="trunk" pos="0 0 .1">
        <joint type="free"/>
        <geom type="box" size=".05 .05 .003" pos="0 0 0" rgba="1 0 0 1"/>
        
      <body name="leg1_l1" pos=".03 -.05 0">
          <joint name="leg1_j1" type="hinge" axis="1 0 0" pos="0 0 0" stiffness="{k:e}" damping="{b:e}" limited="true" range="-179 179"/>
          <geom type="box" size=".02 .02 .001" pos="0 -.02 0" rgba="1 0 0 1"/>
          <body name="leg1_l2" pos=".02 0 0">
            <joint name="leg1_j2" type="hinge" axis="0 1 0"  stiffness="{k:e}" damping="{b:e}"  limited="true" range="-179 179"/>
            <geom type="box" size=".01 .02 .001" pos=".01 -.02 0" rgba="1 0 0 1"/>
            <geom type="box" size=".02 .01 .001" pos="-.02 -.03 0" rgba="1 0 0 1"/>
            <body name="leg1_l4" pos=".02 0 0">
              <joint name="leg1_j3" type="hinge" axis="0 1 0"  stiffness="{k:e}" damping="{b:e}"  limited="true" range="-179 179"/>
              <geom type="box" size=".01 .02 .001" pos=".01 -.02 0" rgba="1 0 0 1"/>
            </body>
          </body>
          <body name="leg1_l3" pos="-.02 0 0">
            <joint name="leg1_j4" type="hinge" axis="0 1 0"  stiffness="{k:e}" damping="{b:e}"  limited="true" range="-179 179"/>
            <geom type="box" size=".01 .02 .001" pos="-.01 -.02 0" rgba="1 0 0 1"/>
            <body name="leg1_l4_2" pos="-.02 0 0">
              <joint name="leg1_j5" type="hinge" axis="0 1 0"  stiffness="{k:e}" damping="{b:e}"  limited="true" range="-179 179"/>
              <geom type="box" size=".01 .02 .001" pos="-.01 -.02 0" rgba="1 0 0 1"/>
            </body>
          </body>
      </body>

      <body name="leg2_l1" pos=".03 .05 0">
          <joint name="leg2_j1" type="hinge" axis="-1 0 0" pos="0 0 0" stiffness="{k:e}" damping="{b:e}" limited="true" range="-179 179"/>
          <geom type="box" size=".02 .02 .001" pos="0 .02 0" rgba="1 0 0 1"/>
          <body name="leg2_l2" pos=".02 0 0">
            <joint name="leg2_j2" type="hinge" axis="0 1 0"  stiffness="{k:e}" damping="{b:e}"  limited="true" range="-179 179"/>
            <geom type="box" size=".01 .02 .001" pos=".01 .02 0" rgba="1 0 0 1"/>
            <geom type="box" size=".02 .01 .001" pos="-.02 .03 0" rgba="1 0 0 1"/>
            <body name="leg2_l4" pos=".02 0 0">
              <joint name="leg2_j3" type="hinge" axis="0 1 0"  stiffness="{k:e}" damping="{b:e}"  limited="true" range="-179 179"/>
              <geom type="box" size=".01 .02 .001" pos=".01 .02 0" rgba="1 0 0 1"/>
            </body>
          </body>
          <body name="leg2_l3" pos="-.02 0 0">
            <joint name="leg2_j4" type="hinge" axis="0 1 0"  stiffness="{k:e}" damping="{b:e}"  limited="true" range="-179 179"/>
            <geom type="box" size=".01 .02 .001" pos="-.01 .02 0" rgba="1 0 0 1"/>
            <body name="leg2_l4_2" pos="-.02 0 0">
              <joint name="leg2_j5" type="hinge" axis="0 1 0"  stiffness="{k:e}" damping="{b:e}"  limited="true" range="-179 179"/>
              <geom type="box" size=".01 .02 .001" pos="-.01 .02 0" rgba="1 0 0 1"/>
            </body>
          </body>
      </body>

    </body>
  </worldbody>


  <actuator>
    <position name="m1" joint="leg1_j1"/>
    <position name="m2" joint="leg1_j2"/>
    <position name="m3" joint="leg2_j1"/>
    <position name="m4" joint="leg2_j2"/>
  </actuator>
  
  <equality>
    <weld name="weld1" active="true" body1="leg1_l4" body2="leg1_l4_2" anchor="-.02 0 0" relpose=".02 0 0 1 0 0 0" solimp=".90 .99 .01" solref=".001 1"/>
    <weld name="weld2" active="true" body1="leg2_l4" body2="leg2_l4_2" anchor="-.02 0 0" relpose=".02 0 0 1 0 0 0" solimp=".90 .99 .01" solref=".001 1"/>
  </equality>  
</mujoco>
"""

In [6]:
def gen_controller(f,A00,A11,t0_1,t0_2,t0_3,t0_4,b1,b2,b3,b4):
        
    def my_controller(model, data):
        t = data.time
        if t<1:
          A0=0
          A1=0
        elif t<2:
          A0 = A00*(t-1)/(2-1)
          A1 = A11*(t-1)/(2-1)
        else:
          A0 = A00
          A1 = A11
        j1 = A0*math.sin(2*math.pi*(f*t-t0_1))+b1
        j2 = A1*math.sin(2*math.pi*(f*t-t0_2))+b2
        j3 = A0*math.sin(2*math.pi*(f*t-t0_3))+b3
        j4 = A1*math.sin(2*math.pi*(f*t-t0_4))+b4
        data.ctrl = [j1,j2,j3,j4]
        return
    return my_controller

In [7]:
def run_sim(model,data,my_controller,render=True):
    mujoco.set_mjcb_control(my_controller)
    frames = []
    mujoco.mj_resetData(model,data)
    data.qpos = qpos

    while data.time < duration:
        mujoco.mj_step(model, data)
        if render:
            if len(frames) < data.time * framerate:
                renderer.update_scene(data,camera='target')
                pixels = renderer.render()
                frames.append(pixels)

    mujoco.set_mjcb_control(None)
    
    
    if render:
        # Simulate and display video.
        media.show_video(frames, fps=framerate,width=width,height=height)

In [8]:
if animal=='beetle':
    xml = beetle_template.format(width=width,height=height,k=1e-3,b=1e-4)
elif animal == 'turtle':
    xml = turtle_template.format(width=width,height=height,k=1e-3,b=1e-4)


model = mujoco.MjModel.from_xml_string(xml)
data = mujoco.MjData(model)
renderer = mujoco.Renderer(model,width=width,height=height)


duration = 10  # (seconds)
framerate = 30  # (Hz)

if animal=='beetle':
    A00 = 30 * math.pi/180
    A11 = 30 * math.pi/180
    f = 1
    t0_1 = 0
    t0_2 = .25
    t0_3 = .5
    t0_4 = .75
    b1 = -90*math.pi/180
    b2 = 90*math.pi/180
    b3 = -90*math.pi/180
    b4 = 90*math.pi/180
elif animal=='turtle':
    A00 = 30 * math.pi/180
    A11 = 30 * math.pi/180
    f = 1
    t0_1 = 0
    t0_2 = .25
    t0_3 = .5
    t0_4 = .75
    b1 = -30*math.pi/180
    b2 = -90*math.pi/180
    b3 = -30*math.pi/180
    b4 = -90*math.pi/180

zero_rotation = numpy.array([1,0,0,0])

if animal=='beetle':
    qpos1 = numpy.array([0,0,.25])
    qpos2 = numpy.array([-90,90,90,-90,-90])* math.pi/180
    qpos3 = numpy.array([-90,90,90,-90,-90])* math.pi/180
    qpos = numpy.hstack([qpos1,zero_rotation,qpos2,qpos3])
elif animal=='turtle':
    qpos1 = numpy.array([0,0,.25])
    qpos2 = numpy.array([0,-90,-90,90,90])* math.pi/180
    qpos3 = numpy.array([0,-90,-90,90,90])* math.pi/180    
    qpos = numpy.hstack([qpos1,zero_rotation,qpos2,qpos3])
qpos

my_controller = gen_controller(f,A00,A11,t0_1,t0_2,t0_3,t0_4,b1,b2,b3,b4)



In [9]:
run_sim(model,data,my_controller,True)

This browser does not support the video tag.


In [10]:
data.qpos

array([ 0.10610106, -0.01260261,  0.01197938,  0.99522792,  0.0112547 ,
       -0.07660667, -0.05938128, -0.52587085, -2.09763578, -1.03660692,
        1.04426181,  2.09030927, -0.52899322, -1.03714197, -2.09736761,
        2.10402548,  1.03031696])

In [11]:
data.tendon

<bound method PyCapsule.tendon of <mujoco._structs.MjData object at 0x7c51f194ad70>>